In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

from gluonts.time_feature import (
    get_lags_for_frequency, 
    time_features_from_frequency_str
)

from transformers import TimeSeriesTransformerConfig

from src.utils import create_test_dataset
from src import ts_transformer as tsf

# Split dataset

In [ ]:
# Assumes pandas.DataFrame <data> in the following format:
# | target | start | feat_static_cat | feat_dynamic_real | item_id |

In [ ]:
context_length = 24
prediction_length = 48

freq = '1H'

transformer_config = {
    'prediction_length': prediction_length,
    'context_length': prediction_length,
    'num_static_categorical_features': 3,
    'cardinality': [52,3,4],
    'embedding_dimension': [2,2,2],
    'encoder_layers': 4,
    'decoder_layers': 4,
    'd_model': 32,
}

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)

#need to split the original series into series of length context+prediction
test_df = test_df.reset_index(drop=True)
new_test_df = create_test_dataset(test_df, context_length+prediction_length)

#get a fraction
frac_test_df = new_test_df.sample(frac=0.1, random_state=42)

# Training

In [ ]:
batch_size = 32
num_batches_per_epoch = 16
epochs = 400

In [ ]:
transformer, train_dataloader, test_dataloader, test_data = tsf.setup_training(
    train_df=train_df,
    test_df=frac_test_df,
    freq=freq,
    batch_size = batch_size,
    num_batches_per_epoch = num_batches_per_epoch,
    transformer_config=transformer_config
)

In [ ]:
transformer, list_loss = tsf.train(transformer, train_dataloader, epochs)

In [ ]:
plt.plot(list_loss)

# Evaluate

In [ ]:
mase, smape = tsf.evaluate(
    test_data,
    forecasts,
    transformer_config['prediction_length'],
    freq)

print(f"MASE: {np.mean(mase)}")

print(f"sMAPE: {np.mean(smape)}")

In [ ]:
plt.scatter(mase, smape, alpha=0.3)
plt.xlabel("MASE")
plt.ylabel("sMAPE")
plt.show()

In [ ]:
plot(17, test_data, forecasts, prediction_length)